In [1]:
import pandas as pd
import os
from sodapy import Socrata

file_path = "./Fire_Department_Calls_for_Service.csv"
if os.path.isfile(file_path):
    fire_df = pd.read_csv(file_path)
else:
    # if file does not exist, then get the data. This may take a long time (IE 30mins)
    client = Socrata("data.sfgov.org", None)
    # Get 1 million results
    amount = (10**3)*1000
    results = client.get("nuek-vuh3", limit=(amount))
    fire_df = pd.DataFrame.from_records(results)
    fire_df.to_csv(file_path)
    fire_df.head()    

C:\Users\Sean4\AppData\Local\Temp\ipykernel_28812\1785054242.py:7: DtypeWarning: Columns (19,20,25,30) have mixed types. Specify dtype option on import or set low_memory=False.
  fire_df = pd.read_csv(file_path)


In [4]:
from datetime import datetime
# import matplotlib.pyplot as plt
from meteostat import Point, Daily
import re

new_data = {"tavg":[],  "tmin":[],  "tmax":[], "prcp":[], "snow":[], "wdir":[], "wspd":[], "wpgt":[], "pres":[], "tsun":[]}
count = 0
for index, row in fire_df.iterrows():
    try:
        count += 1
        if count % 100000 == 0:
            print(count)
        date = row["Call Date"].split("/")
        start = datetime(int(date[2]), int(date[0]), int(date[1]))
        end = start
        location = str(row["case_location"])
        location = re.split(r'\(| |\)', location)
        location = Point(float(location[3]), float(location[2]))
        data = Daily(location, start, end)
        data = data.fetch()
        for keys in new_data:
            new_data[keys].append(data[keys][0])
    except Exception as e:
        for keys in new_data:
            new_data[keys].append(None)
for keys in new_data:
    fire_df[keys] = new_data[keys]


KeyboardInterrupt: 

In [ ]:
fire_df.drop(fire_df.filter(regex="Unname"),axis=1, inplace=True)
fire_df.to_csv("./Fire_Department_Calls_for_Service_weather.csv", index=False)